# start

In [4]:
trntmstmp=1584412344
valtmstmp=1585069785
import datetime
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (trntmstmp, valtmstmp)])

grand_total=1.5e8
MIN_TM_TRN=1580947200
MIN_TM_TST=1581552000
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (MIN_TM_TRN, MIN_TM_TST)])


CHNKSZ=1e6
POST_RATE_WANTED=0.1

['2020-03-17 02:32:24', '2020-03-24 17:09:45']
['2020-02-06 00:00:00', '2020-02-13 00:00:00']


# setup

In [114]:
import pandas as pd
pd.set_option('display.max_columns', 500)

SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'

import torch
from transformers import *
import torch
device=torch.device('cpu')

pretrained_weights='bert-base-multilingual-cased'
model = BertModel.from_pretrained(pretrained_weights, output_hidden_states=True, num_labels=4)
tokenizer = BertTokenizer.from_pretrained(pretrained_weights, do_lower_case=False)


In [115]:
cols=[
'toks',
'hshtgs',
'twtid',
'media',
'links',
'domns',
'twttyp',
'lang',
'tm',

'u1id',
'u1_fllwer_cnt',
'u1_fllwng_cnt',
'u1_vrfed',
'u1_create_tm',

'u2id',
'u2_fllwer_cnt',
'u2_fllwng_cnt',
'u2_vrfed',
'u2_create_tm',

'u1_fllw_u2',
'reply_tm',
'retwt_tm',
'retwt_cmmnt_tm',
'like_tm',
]
cols_cat = ['twttyp','lang']
cols_val = cols[:-4]
cols_tgt_tmstmp=[
    'retwt_tm',
    'reply_tm',
    'like_tm',
    'retwt_cmmnt_tm',
]
cols_tgt=[o.split('_tm')[0] for o in cols_tgt_tmstmp]
tgts             = ['Retweet','Reply','Like','RTwCmnt',]
assert cols_tgt == ['retwt',  'reply','like','retwt_cmmnt',]
ntgts=len(tgts)


tgt2col=dict(zip(tgts,cols_tgt))
tgt2col

{'Retweet': 'retwt',
 'Reply': 'reply',
 'Like': 'like',
 'RTwCmnt': 'retwt_cmmnt'}

In [243]:
%%time
dftrn=pd.read_csv(
    f'{p_in}/trn_{trntmstmp}.tsv',
    sep='\x01', header=None, 
    encoding='utf-8', 
    names=cols, 
    nrows=1e4
)

CPU times: user 67.8 ms, sys: 11.6 ms, total: 79.4 ms
Wall time: 78.4 ms


In [268]:
x=dftrn.toks[0]

In [269]:
tokids=list(map(int, x.split('\t')))
tokids

[101, 6417, 3410, 3398, 3184, 1909, 56910, 16838, 82904, 1901, 100, 102]

In [270]:
" ".join(tokenizer.convert_ids_to_tokens(tokids))

'[CLS] 美 容 室 変 え ##ど ##き ##かな 〜 [UNK] [SEP]'

In [271]:
maxlen=8
len(tokids)

12

In [272]:
tokids=tokids + [0] * (maxlen-len(tokids))
tokids

[101, 6417, 3410, 3398, 3184, 1909, 56910, 16838, 82904, 1901, 100, 102]

In [274]:
tokids=tokids[:maxlen-1]+[tokenizer.sep_token_id,]

In [275]:
tokids

[101, 6417, 3410, 3398, 3184, 1909, 56910, 102]

In [252]:
tokids=tokids + [0] * (maxlen-len(tokids))

In [253]:
tokids=tokids

[101,
 6417,
 3410,
 3398,
 3184,
 1909,
 56910,
 16838,
 82904,
 1901,
 100,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [239]:
dftrn.toks[0]+

[101, 6417, 3410, 3398, 3184, 1909, 56910, 16838, 82904, 1901, 100, 102]

In [238]:
tokenizer.pad_token, tokenizer.pad_token_id

('[PAD]', 0)

In [199]:
lens_toks=dftrn.toks.apply(len)

In [200]:
max(lens_toks)

750

In [221]:
toks=dftrn.sample().toks.values[0]

" ".join(tokenizer.convert_ids_to_tokens(toks))

"[CLS] RT @ Phantom ##A ##rti ##fice : i saw this on red ##dit and haven ' t seen something more true https : / / t . co / k ##7 ##c ##J ##O ##Z ##ht ##X ##x [SEP]"

In [224]:
toks=[int(o) for o in toks]

# https://huggingface.co/transformers/model_doc/bert.html#bertmodel
model=model.eval()
input_ids = torch.tensor([toks])  
with torch.no_grad():
    last_hidden_state, pooler_output, hidden_states = model(input_ids)
    avg_pool = torch.mean(last_hidden_state,1)
    max_pool,_ = torch.max(last_hidden_state,1)
last_hidden_state.shape,pooler_output.shape,len(hidden_states), avg_pool.shape, max_pool.shape

(torch.Size([1, 40, 768]),
 torch.Size([1, 768]),
 13,
 torch.Size([1, 768]),
 torch.Size([1, 768]))